In [1]:
from fastapi import FastAPI
from fastapi.responses import HTMLResponse
import pandas as pd
import datetime
import locale
from data import df as df
import re
from unidecode import unidecode
from IPython.display import display
from pandas import DataFrame

In [2]:
df.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,iso,countrie_name,iso_lang,lang_name,release_year,return,character,actor_id,actor_name,Director
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-October-30,373554033.0,81.0,Released,NaN,...,US,United States of America,en,English,1995,12.451801,"Woody (voice), Buzz Lightyear (voice), Mr. Pot...","31, 12898, 7167, 12899, 12900, 7907, 8873, 111...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-December-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,US,United States of America,"en, fr","English, Français",1995,4.043035,"Alan Parrish, Samuel Alan Parrish / Van Pelt, ...","2157, 8537, 205, 145151, 5149, 10739, 58563, 1...","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-December-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,US,United States of America,en,English,1995,0.000000,"Max Goldman, John Gustafson, Ariel Gustafson, ...","6837, 3151, 13567, 16757, 589, 16523, 7166","Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-December-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,US,United States of America,en,English,1995,5.090760,"""Savannah Vannah Jackson"", ""Bernadine Bernie H...","8851, 9780, 18284, 51359, 66804, 352, 87118, 3...","Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-February-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,US,United States of America,en,English,1995,0.000000,"George Banks, Nina Banks, Franck Eggelhoffer, ...","67773, 3092, 519, 70696, 59222, 18793, 14592, ...","Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer


In [3]:
# Instance of FastApi
app = FastAPI()

In [6]:
meses = {
        "enero": "January",
        "febrero": "February",
        "marzo": "March",
        "abril": "April",
        "mayo": "May",
        "junio": "June",
        "julio": "July",
        "agosto": "August",
        "septiembre": "September",
        "octubre": "October",
        "noviembre": "November",
        "diciembre": "December"
    }

In [7]:
# 1 function number of films per month

@app.get("/cantidad_filmaciones_mes/{mes}")
def cantidad_filmaciones_mes(mes: str):
    # Convertir el mes en español a minúsculas y eliminar espacios adicionales
    mes = mes.lower().strip()

    # Verificar si el mes está en el diccionario
    if mes in meses:
        mes_en_ingles = meses[mes]
        cantidad = len(df[df["release_date"].str.contains(mes_en_ingles, case=False)])
        return f"{cantidad} películas fueron estrenadas en el mes de {mes}"
    else:
        return f"No se reconoce el mes '{mes}'. Por favor, ingresa un mes válido en español."

In [8]:
cantidad_filmaciones_mes('mayo')

'3341 películas fueron estrenadas en el mes de mayo'

In [6]:
@app.get("/cantidad_filmaciones_dia/{dia}")
def cantidad_filmaciones_dia(dia: str):
    valid_days = ['lunes', 'martes', 'miercoles', 'jueves', 'viernes', 'sabado', 'domingo']
    dia_unaccented = unidecode(dia.lower().strip().replace(" ", ""))
    
    if dia_unaccented not in valid_days:
        return 'No ha ingresado un día de la semana válido en idioma español'
    
    df['release_day'] = pd.to_datetime(df['release_date'], format="%Y-%B-%d").dt.strftime('%A')
    films = len(df[df["release_day"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.contains(dia_unaccented, case=False)])
    
    return f"{films} películas fueron estrenadas en los días {dia}"


In [7]:
cantidad_filmaciones_dia('jueves')

'7531 películas fueron estrenadas en los días jueves'

In [8]:
@app.get("/score_title/{film_title}")
def score_title(film_title: str):
    films = df[df["title"].str.lower() == film_title.lower()]
    if films.empty:
        return "No se encontró la filmación especificada."
    
    ordered_films = films.sort_values(by="release_year")
    
    answers = []
    for index, row in ordered_films.iterrows():
        title = row["title"]
        year_released = row["release_year"]
        score = row["popularity"]
        answer = "Película: ", title, "Año estreno: ", year_released, "Score/Popularidad",  score, "\n"
        answers.append(answer)
    
    return answers



In [10]:
def score_title(film_title: str):
    films = df[df["title"].str.lower() == film_title.lower()]
    if films.empty:
        return "No se encontró la filmación especificada."

    ordered_films = films.sort_values(by="release_year")

    table_html = "<table>"
    table_html += "<tr><th>Película</th><th>Año estreno</th><th>Score/Popularidad</th></tr>"

    for index, row in ordered_films.iterrows():
        title = row["title"]
        year_released = row["release_year"]
        score = row["popularity"]

        table_html += "<tr>"
        table_html += f"<td>{title}</td>"
        table_html += f"<td>{year_released}</td>"
        table_html += f"<td>{score}</td>"
        table_html += "</tr>"

    table_html += "</table>"

    return HTMLResponse(content=table_html, status_code=200)

In [11]:
score_title('cinderella')

In [ ]:
pd.set_option('display.max_columns', None)

In [18]:
@app.get("/score_title/{film_title}")
def votos_titulo(titulo_de_la_filmacion: str):
    filmacion = df[df["title"].str.lower() == titulo_de_la_filmacion.lower()]
    
    if filmacion.empty:
        return "No se encontró la filmación especificada."
    
    if filmacion["vote_count"].values[0] < 2000:
        return "La película no cuenta con al menos 2000 valoraciones."
    
    titulo = filmacion["title"].values[0]
    votos = filmacion["vote_count"].values[0]
    promedio = filmacion["vote_average"].values[0]
    
    respuesta = f"La película {titulo} fue estrenada en el año {filmacion['release_year'].values[0]}. La misma cuenta con un total de {votos} valoraciones, con un promedio de {promedio}."
    return respuesta


In [20]:
votos_titulo('12 angry men')

'La película 12 Angry Men fue estrenada en el año 1957. La misma cuenta con un total de 2130.0 valoraciones, con un promedio de 8.2.'

In [12]:
def get_actor(actor_name):
    actor_films = df[df["actor_name"].str.contains(fr"\b{actor_name}\b", case=False, regex=True, na=False)]
    
    if actor_films.empty:
        return "No se encontró el actor especificado."

    film_count = len(actor_films)
    total_return = actor_films["return"].sum()
    average_return = total_return / film_count

    respuesta = f"El actor {actor_name} ha participado en {film_count} filmaciones. Ha conseguido un retorno total de {total_return} con un promedio de {average_return} por filmación."
    return respuesta


In [17]:
get_actor('woody allen')

'El actor woody allen ha participado en 56 filmaciones. Ha conseguido un retorno total de 75.17772737111272 con un promedio de 1.3424594173412987 por filmación.'

In [15]:
def get_director(director_name):
    director_movies = df[df['Director'].str.contains(fr"\b{director_name}\b", case=False, regex=True, na=False)]
    if director_movies.empty:
        return {"message": "No se encontró el director especificado."}
    film_count = len(director_movies)
    total_return = director_movies['return'].sum()
    answer = f"El director {director_name} ha dirigido {film_count} película(s). Con un ROI total de {total_return}"
    movies_data = director_movies[['Director', 'title', 'release_date', 'return', 'budget', 'revenue']]

    
    return {"answer": answer, "movies_data": movies_data}

In [16]:
get_director('woody')

{'answer': 'El director woody ha dirigido 49 película(s). Con un ROI total de 96.88784128640854',
 'movies_data':                                                 Director  \
 51                                           Woody Allen   
 343                                          Woody Allen   
 487                                          Woody Allen   
 1032                                         Woody Allen   
 1050                                         Woody Allen   
 1051                                         Woody Allen   
 1191                                         Woody Allen   
 1204                                         Woody Allen   
 1625                                         Woody Allen   
 1858                                         Woody Allen   
 1960                                         Woody Allen   
 2181                                         Woody Allen   
 2247                                         Woody Allen   
 2642                            

In [ ]:
def get_director(director_name: str):
    # Filtrar las películas por director
    director_movies = df[df['Director'].str.contains(fr"\b{director_name}\b", case=False, regex=True, na=False)]

    if director_movies.empty:
        return "No se encontró el director especificado."

    film_count = len(director_movies)
    total_return = director_movies['return'].sum()
    answer = f"El director {director_name} ha dirigido {film_count} pelicula(s). Con un ROI total de {total_return}"

    # Generar la representación HTML de la tabla
    movies_table = director_movies[['Director', 'title', 'release_date', 'return', 'budget', 'revenue']]
    movies_table_html = movies_table.to_html()

    # Crear la respuesta HTML con el texto y la tabla generada
    response_content = f"<h1>{answer}</h1>{movies_table_html}"
    return HTMLResponse(content=response_content)

In [ ]:
get_director('arman')

In [ ]:
def get_director(director_name):
    director_films = df[df["Director"].str.contains(fr"\b{director_name}\b", case=False, regex=True, na=False)]
    
    if director_films.empty:
        return "No se encontró el director especificado."

    films_info = director_films[["title", "release_date", "return", "budget", "revenue"]]
    respuesta = f"El director {director_name} ha dirigido las siguientes películas:\n\n"
    for _, row in films_info.iterrows():
        titulo = row["title"]
        fecha_lanzamiento = row["release_date"]
        retorno_individual = row["return"]
        costo = row["budget"]
        ganancia = row["revenue"]
        respuesta += f"Título: {titulo}\nFecha de lanzamiento: {fecha_lanzamiento}\nRetorno individual: {retorno_individual}\nCosto: {costo}\nGanancia: {ganancia}\n\n"

    return respuesta


In [21]:
@app.get("/votos_titulo/{titulo_de_la_filmacion}")
def votos_titulo(titulo_de_la_filmacion: str):
    filmaciones = df[df["title"].str.lower() == titulo_de_la_filmacion.lower()]
    
    if filmaciones.empty:
        return "No se encontró la filmación especificada."
    
    respuestas = []
    
    for _, row in filmaciones.iterrows():
        cantidad_votos = row["vote_count"]
        
        if cantidad_votos < 2000:
            continue  # Saltar esta película si no cumple con la cantidad mínima de votos
        
        titulo = row["title"]
        promedio_votos = row["vote_average"]
        respuesta = f"Título: {titulo}<br/>Cantidad de votos: {cantidad_votos}<br/>Valor promedio de votaciones: {promedio_votos}<br/><br/>"
        respuestas.append(respuesta)
    
    if not respuestas:
        return "No se encontraron filmaciones que cumplan con la cantidad mínima de votos requerida (2000 votos)."
    
    return "\n".join(respuestas)


In [32]:
votos_titulo('Cinderella')

'Título: Cinderella<br/>Cantidad de votos: 2426.0<br/>Valor promedio de votaciones: 6.7<br/><br/>'

In [28]:
df.title

0                          Toy Story
1                            Jumanji
2                   Grumpier Old Men
3                  Waiting to Exhale
4        Father of the Bride Part II
                    ...             
45418                     Robin Hood
45419            Century of Birthing
45420                       Betrayal
45421               Satan Triumphant
45422                       Queerama
Name: title, Length: 45423, dtype: object